In [93]:
import numpy
from urllib.request import urlopen
from collections import defaultdict
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
import matplotlib.pyplot as plt
import scipy
import scipy.optimize
import random
import copy
import gzip
import json
import string

def parseData(fname):
    for l in open(fname):
        yield eval(l)
def feature_label_default(data):
    return "fit"

def feature_label(data):
    return data["fit"]
def feature_X(data,encoding):
    hotencoding=encoding[catID[data['category']]]
    return [1,data['height'],data['weight'],int(data['rating']),int(data['size'])]+hotencoding
def str_to_inch(s):
    s = s.replace('\"', '')
    height=s.split("\'")
    return int(height[0])*12+int(height[1])
def str_to_weight(s):
    s = s.replace('lbs', '')
    return int(s)




test_data = list(parseData("modcloth_final_data.json"))
#random.shuffle(data)
print(test_data[:10])




[{'item_id': '123373', 'waist': '29', 'size': 7, 'quality': 5, 'cup size': 'd', 'hips': '38', 'bra size': '34', 'category': 'new', 'bust': '36', 'height': '5ft 6in', 'user_name': 'Emily', 'length': 'just right', 'fit': 'small', 'user_id': '991571'}, {'item_id': '123373', 'waist': '31', 'size': 13, 'quality': 3, 'cup size': 'b', 'hips': '30', 'bra size': '36', 'category': 'new', 'length': 'just right', 'height': '5ft 2in', 'user_name': 'sydneybraden2001', 'fit': 'small', 'user_id': '587883'}, {'item_id': '123373', 'waist': '30', 'size': 7, 'quality': 2, 'cup size': 'b', 'shoe size': '9.00', 'bra size': '32', 'category': 'new', 'length': 'slightly long', 'height': '5ft 7in', 'user_name': 'Ugggh', 'fit': 'small', 'user_id': '395665'}, {'item_id': '123373', 'category': 'new', 'size': 21, 'quality': 5, 'user_name': 'alexmeyer626', 'length': 'just right', 'fit': 'fit', 'cup size': 'dd/e', 'user_id': '875643'}, {'item_id': '123373', 'category': 'new', 'size': 18, 'quality': 5, 'user_name': 'd

In [94]:
#SET UP CATEGORY FOR data
categoryCounts = defaultdict(int)
for d in data1:
    categoryCounts[d['category']] += 1
categories = [c for c in categoryCounts]
catID = dict(zip(list(categories),range(len(categories))))
print(categoryCounts)

hot_encoding={}                                        #hot encoding of 0-41
for i in range(len(categories)):
    encoding=[0]*len(categories)
    encoding[i]=1
    hot_encoding[i]=encoding
    
#print(hot_encoding)

defaultdict(<class 'int'>, {'dress': 72894, 'down': 387, 'gown': 34445, 'sheath': 15189, 'top': 3923, 'shift': 4136, 'maxi': 2678, 'skirt': 1241, 'jumpsuit': 4159, 'sweater': 948, 'romper': 2517, 'jacket': 1909, 'mini': 1438, 'blazer': 633, 'cardigan': 203, 'pant': 85, 'blouson': 11, 'coat': 792, 'tank': 145, 'blouse': 504, 'shirtdress': 583, 'ballgown': 13, 'hoodie': 11, 'kimono': 24, 'pullover': 46, 'culottes': 163, 'vest': 222, 'turtleneck': 23, 'shirt': 220, 'pants': 340, 'frock': 170, 'trouser': 50, 'tunic': 137, 'suit': 108, 'parka': 13, 'bomber': 103, 'legging': 81, 'sweatshirt': 104, 'midi': 46, 'jeans': 4, 'duster': 12, 'poncho': 34, 't-shirt': 11, 'leggings': 95, 'peacoat': 33, 'print': 90, 'cami': 12, 'knit': 42, 'culotte': 60, 'overalls': 6, 'sweatershirt': 3, 'cape': 69, 'tee': 20, 'trousers': 12, 'caftan': 4, 'skort': 4, 'for': 5, 'jogger': 4, 'skirts': 5, 'trench': 20, 'kaftan': 13, 'combo': 8, 'henley': 7, 'tight': 13, 'sweatpants': 1, 'crewneck': 1, 'buttondown': 1, 'o

In [106]:
def TruePos(pred,y,label):
    res=0
    for pred_, y_ in zip(pred, y):
        if pred_ == y_ and y_== label:
            
            res+=1
    return res

def FalsePos(pred,y,label):
    res=0
    for pred_, y_ in zip(pred, y):
        if pred_ == label and  y_ != label:
            
            res+=1
    return res

def TrueNeg(pred,y,label):
    res=0
    for pred_, y_ in zip(pred, y):
        if  pred_ != label and label != y_:
            
                res+=1
    return res

def FalseNeg(pred,y,label):
    res=0
    for pred_, y_ in zip(pred, y):
        if label!= pred_ and y_==label:
            
            res+=1
    return res
def BER(pred,train_y,label):
    TP=TruePos(pred,train_y,label)
    FP=FalsePos(pred,train_y,label)
    TN=TrueNeg(pred,train_y,label)
    FN=FalseNeg(pred,train_y,label)
    TPR= TP/(TP+FN)
    TNR= TN/(TN+FP)
    FPR= FP/(FP+TN)
    FNR=FN/(FN+TP)
    if (TP+FP)==0:
        PERCISION=0
    else:
        PERCISION= TP/(TP+FP)
    
    #BER= 1-0.5*(TPR+TNR)
    BER= 0.5*(FPR+FNR)
    print("Number of time predicted",label," correct :", TP)
    print("Number of time predicted",label," Wrong :", FP)
    print("Number of time predicted different than actual",label,"  :", FN)
    
    print("RECALL ",label," is :", TPR)
    print("TNR ",label," is :", TNR)
    print("PERCISION ",label," is :", PERCISION)
    print("FPR ",label," is :", FPR)
    print("BER ",label," is :", BER)


In [96]:
#f = open('renttherunway_final_data.json', "r")
data = list([json.loads(line) for line in open('renttherunway_final_data.json', 'r')])
    
data1=[]
data_test=[]
for i in data:
    if 'weight' in i and 'bust size' in i and 'height' in i and 'rating' in i and 'review_text' in i:
        if i['rating'] != None:
            i['weight']=str_to_weight(i['weight'])
            i['height']=str_to_inch(i['height'])
            data1.append(i)

for i in test_data:
    if 'weight' in i and 'bust size' in i and 'height' in i and 'rating' in i and 'review_text' in i:
        i['weight']=str_to_weight(i['weight'])
        i['height']=str_to_inch(i['height'])
        data_test.append(i)

In [97]:
#print(len(data_train),len(data))
print(len(data_test),len(test_data))    #different data sets cannot be test with training set

random.shuffle(data1)
data_train=data1[:len(data1)//10*9]
data_test=data1[len(data1)//10*9:]
print(len(data_train),len(data_test),len(data1))

0 82790
136152 15132 151284


In [98]:
print(type(data[0]))
for key in data[0]:
    print(key,':',data[0][key])



string1= "5'8\""
#print(data_train[1]['height'])
#print(str_to_inch(data_train[1]['height']))
    

<class 'dict'>
fit : fit
user_id : 420272
bust size : 34d
item_id : 2260466
weight : 137
rating : 10
rented for : vacation
review_text : An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.
body type : hourglass
review_summary : So many compliments!
category : romper
height : 68
size : 14
age : 28
review_date : April 20, 2016


In [99]:
#print(test_data[:1])
#l=[feature_height(d) for d in data_train]
dic= defaultdict(int)
#print(l[0])
#for i in l:                                       #num of fit/small/large
#    j=str(i)
#    dic[j]+=1

p=[d['category'] for d in data_train]             #num of items per category
#for i in p:
#    j=str(i)
#    dic[j]+=1
    
print(dic)

defaultdict(<class 'int'>, {})


In [100]:
#possible missing values:

#bust size
#weight
#rating
#review_text
#height


In [101]:
usersPeritem = defaultdict(set)
itemPerUser = defaultdict(set)
for d in data:
    user,item = d['user_id'], d['item_id']
    usersPeritem[item].add(user)
    itemPerUser[user].add(item)
    






In [102]:
#for i in itemPerUser:
   # print(len(itemPerUser[i]))
#print(len(itemPerUser))          #

In [103]:
X_train=[feature_X(d,hot_encoding) for d in data_train]
X_test=[feature_X(d,hot_encoding) for d in data_test]

Y_train=[feature_label(d) for d in data_train]
Y_test=[feature_label(d) for d in data_test]

In [108]:
baseline=numpy.array([feature_label_default(d) for d in data_test])
print(len(baseline),len(Y_test))
correct_base = baseline==Y_test

print("accuracy feature:weight,height,rating", sum(correct_base)/len(correct_base))
#print(len(correct1))
print('done..')

BER(baseline,Y_test,'fit')
BER(baseline,Y_test,'large')
BER(baseline,Y_test,'small')

15132 15132
accuracy feature:weight,height,rating 0.732355273592387
done..
Number of time predicted fit  correct : 11082
Number of time predicted fit  Wrong : 4050
Number of time predicted different than actual fit   : 0
RECALL  fit  is : 1.0
TNR  fit  is : 0.0
PERCISION  fit  is : 0.732355273592387
FPR  fit  is : 1.0
BER  fit  is : 0.5
Number of time predicted large  correct : 0
Number of time predicted large  Wrong : 0
Number of time predicted different than actual large   : 1972
RECALL  large  is : 0.0
TNR  large  is : 1.0
PERCISION  large  is : 0
FPR  large  is : 0.0
BER  large  is : 0.5
Number of time predicted small  correct : 0
Number of time predicted small  Wrong : 0
Number of time predicted different than actual small   : 2078
RECALL  small  is : 0.0
TNR  small  is : 1.0
PERCISION  small  is : 0
FPR  small  is : 0.0
BER  small  is : 0.5


In [109]:
mod = linear_model.LogisticRegression(C=2.0,)
mod.fit(X_train, Y_train)




/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=2.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [110]:
pred_valid= mod.predict(X_test)
correct1 = pred_valid==Y_test

print("accuracy feature:weight,height,rating", sum(correct1)/len(correct1))
#print(len(correct1))
print('done..')

accuracy feature:weight,height,rating 0.7340074015331747
done..


In [111]:
print("accuracy feature:weight,height,rating", sum(correct1)/len(correct1))
BER(pred_valid,Y_test,'fit')
BER(pred_valid,Y_test,'large')
BER(pred_valid,Y_test,'small')

accuracy feature:weight,height,rating 0.7340074015331747
Number of time predicted fit  correct : 10972
Number of time predicted fit  Wrong : 3805
Number of time predicted different than actual fit   : 110
RECALL  fit  is : 0.9900739938639235
TNR  fit  is : 0.06049382716049383
PERCISION  fit  is : 0.7425052446369358
FPR  fit  is : 0.9395061728395062
BER  fit  is : 0.47471608948779137
Number of time predicted large  correct : 35
Number of time predicted large  Wrong : 76
Number of time predicted different than actual large   : 1937
RECALL  large  is : 0.017748478701825558
TNR  large  is : 0.994224924012158
PERCISION  large  is : 0.3153153153153153
FPR  large  is : 0.005775075987841946
BER  large  is : 0.4940132986430082
Number of time predicted small  correct : 100
Number of time predicted small  Wrong : 144
Number of time predicted different than actual small   : 1978
RECALL  small  is : 0.04812319538017324
TNR  small  is : 0.9889688984219397
PERCISION  small  is : 0.4098360655737705
FP

In [112]:


dic1= defaultdict(int)
dic2=defaultdict(int)

#print(l[0])
for i in pred_valid:                                       #num of fit/small/large
    j=str(i)
    dic1[j]+=1
for i in Y_test:                                       #num of fit/small/large
    j=str(i)
    dic2[j]+=1
    
print(dic1,dic2)

defaultdict(<class 'int'>, {'fit': 14777, 'large': 111, 'small': 244}) defaultdict(<class 'int'>, {'large': 1972, 'fit': 11082, 'small': 2078})


In [113]:
print(type(Y_train[1]))

<class 'str'>


In [114]:
print([feature_label(d) for d in data_train[:5]])

['fit', 'fit', 'fit', 'fit', 'fit']


In [115]:
#try bag of words methods
sp = set(string.punctuation)
wordCount = defaultdict(int)
for d in data1:
    r = ''.join([c for c in d['review_text'].lower() if not c in sp])
    tokens = r.split()
    for w in tokens:
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()



In [116]:
print(len(counts))

44034


In [117]:
from scipy.sparse import lil_matrix
word_num = len(counts)//10
words = [x[1] for x in counts[:word_num]]
from nltk.corpus import stopwords
stopwords = stopwords.words("english")
filterword = [w for w in words if not w in stopwords]
#filterword = [w for w in words]
wordId = dict(zip(filterword, range(len(filterword))))
wordSet = set(filterword)
word_df = dict(zip(filterword, [0]*(len(filterword))))

for d in data1:
    r = ''.join([c for c in d['review_text'].lower() if not c in sp])
    tokens = r.split()
    sets= set(tokens)
    for i in sets:
        if i in filterword:
            word_df[i]+=1
print('done..')

done..


In [118]:
import copy
import math
word_df1= copy.deepcopy(word_df)
N=len(data1)
for i in word_df1:
    word_df1[i]= math.log(N/word_df1[i])
#print(word_df1)

In [119]:
def feature1(datum):
    feat = [0]*len(filterword)
    r = ''.join([c for c in datum['review_text'].lower() if not c in sp])
    tokens = r.split()
    for w in tokens:
        if w in wordSet:
            feat[wordId[w]] += 1
    sets=set(tokens)
    for s in sets:
        if s in wordSet:
            feat[wordId[s]]=feat[wordId[s]]*word_df1[s]

    feat.append(1)
    return feat

nf1 = len(feature1(data1[0]))

X1 = lil_matrix((len(data1), nf1))

for i in range(len(data1)):
    
    x = feature1(data1[i])
    for j in range(nf1):
        if x[j]:
            X1[i,j] = x[j]

print('done..')

done..


In [120]:
Xtrain1 = X1[:len(data1)//10*9]

Xtest1 = X1[len(data1)//10*9:]



mod_ifdif = linear_model.LogisticRegression(C=2.0,random_state=0)
mod_ifdif.fit(Xtrain1, Y_train)



/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=2.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [121]:
pred_valid2= mod_ifdif.predict(Xtest1)
correct2 = pred_valid2==Y_test
print("accuracy feature:bag of words idf", sum(correct2)/len(correct2))


#print(len(correct1))
print('done..')
print('done')

BER(pred_valid2,Y_test,'fit')
BER(pred_valid2,Y_test,'large')
BER(pred_valid2,Y_test,'small')

accuracy feature:bag of words idf 0.7849590272270685
done..
done
Number of time predicted fit  correct : 10391
Number of time predicted fit  Wrong : 2379
Number of time predicted different than actual fit   : 691
RECALL  fit  is : 0.9376466341815557
TNR  fit  is : 0.41259259259259257
PERCISION  fit  is : 0.8137039937353171
FPR  fit  is : 0.5874074074074074
BER  fit  is : 0.32488038661292584
Number of time predicted large  correct : 749
Number of time predicted large  Wrong : 425
Number of time predicted different than actual large   : 1223
RECALL  large  is : 0.37981744421906694
TNR  large  is : 0.9677051671732523
PERCISION  large  is : 0.6379897785349233
FPR  large  is : 0.03229483282674772
BER  large  is : 0.3262386943038404
Number of time predicted small  correct : 738
Number of time predicted small  Wrong : 450
Number of time predicted different than actual small   : 1340
RECALL  small  is : 0.3551491819056785
TNR  small  is : 0.9655278075685614
PERCISION  small  is : 0.62121212121

In [122]:
dic1= defaultdict(int)
dic2=defaultdict(int)

#print(l[0])
for i in pred_valid1:                                       #num of fit/small/large
    j=str(i)
    dic1[j]+=1
for i in Y_test:                                       #num of fit/small/large
    j=str(i)
    dic2[j]+=1
    
print(dic1,dic2)

defaultdict(<class 'int'>, {'fit': 12782, 'large': 1222, 'small': 1128}) defaultdict(<class 'int'>, {'large': 1972, 'fit': 11082, 'small': 2078})


In [123]:
##i fdif result
dic1= defaultdict(int)
dic2=defaultdict(int)
dic3=defaultdict(int)
for i,j in zip(pred_valid1,Y_test):
    if(i==j and i =='small'):
        dic1[i]+=1
    elif (i==j and i =='large'):
        dic1[i]+=1
    elif(i==j and i =='fit'):
        dic1[i]+=1
    elif(i=='small'):
        dic1['predict small but wrong']+=1
    elif(i=='large'):
        dic1['predicted large but wrong']+=1
    else:
        dic1['predict fit but wrong']+=1
print(dic1)

defaultdict(<class 'int'>, {'predict fit but wrong': 3425, 'fit': 9357, 'predicted large but wrong': 1060, 'predict small but wrong': 988, 'large': 162, 'small': 140})


In [124]:
def feature(datum):
    hotencoding=hot_encoding[catID[datum['category']]]
    feat = [0]*len(filterword)
    r = ''.join([c for c in datum['review_text'].lower() if not c in sp])
    tokens = r.split()
    for w in tokens:
        if w in wordSet:
            feat[wordId[w]] += 1
    #feat.append(datum['height'])
    #feat.append(datum['weight'])
    
    feat.append(datum['rating'])
    feat= feat+hotencoding
    feat.append(1)
    return feat

nf = len(feature(data1[0]))

X = lil_matrix((len(data1), nf))

for i in range(len(data1)):
    
    x = feature(data1[i])
    for j in range(nf):
        if x[j]:
            X[i,j] = x[j]

print('done..')

done..


In [125]:
#if(X!=X1):
#    print("false")

In [126]:
Xtrain = X[:len(data1)//10*9]

Xtest = X[len(data1)//10*9:]



mod1 = linear_model.LogisticRegression(C=2.0)
mod1.fit(Xtrain, Y_train)
print('done')

done


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [127]:
pred_valid3= mod1.predict(Xtest)
correct3 = pred_valid3==Y_test
#BER(pred_valid,Y_test)
print("accuracy feature:bag of words", sum(correct3)/len(correct3))
BER(pred_valid3,Y_test,'fit')
BER(pred_valid3,Y_test,'large')
BER(pred_valid3,Y_test,'small')
#print(len(correct1))
print('done..')

accuracy feature:bag of words 0.8018107322231034
Number of time predicted fit  correct : 10545
Number of time predicted fit  Wrong : 2315
Number of time predicted different than actual fit   : 537
RECALL  fit  is : 0.9515430427720628
TNR  fit  is : 0.4283950617283951
PERCISION  fit  is : 0.8199844479004665
FPR  fit  is : 0.571604938271605
BER  fit  is : 0.3100309477497711
Number of time predicted large  correct : 804
Number of time predicted large  Wrong : 321
Number of time predicted different than actual large   : 1168
RECALL  large  is : 0.4077079107505071
TNR  large  is : 0.9756079027355623
PERCISION  large  is : 0.7146666666666667
FPR  large  is : 0.02439209726443769
BER  large  is : 0.3083420932569653
Number of time predicted small  correct : 784
Number of time predicted small  Wrong : 363
Number of time predicted different than actual small   : 1294
RECALL  small  is : 0.37728585178055823
TNR  small  is : 0.9721924314386395
PERCISION  small  is : 0.6835222319093287
FPR  small  i

In [128]:
#FP=FalsePos(pred_valid,Y_test)
#print(pred_valid,Y_test)

In [129]:
for C in [0.01,0.1,1,2,5,10]:
    modC = linear_model.LogisticRegression(C=C,class_weight='balanced')
    modC.fit(Xtrain, Y_train)
    pred_validC= modC.predict(Xtest)
    correctC = pred_validC==Y_test
    #BER(pred_valid,Y_test)
    print("accuracy feature:bag of words,rating and category for C= ",C, sum(correctC)/len(correctC))
    BER(pred_validC,Y_test,'fit')
    BER(pred_validC,Y_test,'large')
    BER(pred_validC,Y_test,'small')
    print('----------------------------------------------------------------------------------------------------')
    



/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


accuracy feature:bag of words,rating and category for C=  0.01 0.7107454401268835
Number of time predicted fit  correct : 8066
Number of time predicted fit  Wrong : 931
Number of time predicted different than actual fit   : 3016
RECALL  fit  is : 0.7278469590326656
TNR  fit  is : 0.7701234567901235
PERCISION  fit  is : 0.8965210625764144
FPR  fit  is : 0.22987654320987655
BER  fit  is : 0.25101479208860544
Number of time predicted large  correct : 1288
Number of time predicted large  Wrong : 1645
Number of time predicted different than actual large   : 684
RECALL  large  is : 0.6531440162271805
TNR  large  is : 0.875
PERCISION  large  is : 0.43914081145584727
FPR  large  is : 0.125
BER  large  is : 0.23592799188640973
Number of time predicted small  correct : 1401
Number of time predicted small  Wrong : 1801
Number of time predicted different than actual small   : 677
RECALL  small  is : 0.6742059672762272
TNR  small  is : 0.8620346254021756
PERCISION  small  is : 0.4375390381011868
FP

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


accuracy feature:bag of words,rating and category for C=  0.1 0.7063838223632039
Number of time predicted fit  correct : 8005
Number of time predicted fit  Wrong : 931
Number of time predicted different than actual fit   : 3077
RECALL  fit  is : 0.722342537448114
TNR  fit  is : 0.7701234567901235
PERCISION  fit  is : 0.8958146821844225
FPR  fit  is : 0.22987654320987655
BER  fit  is : 0.2537670028808813
Number of time predicted large  correct : 1285
Number of time predicted large  Wrong : 1681
Number of time predicted different than actual large   : 687
RECALL  large  is : 0.6516227180527383
TNR  large  is : 0.8722644376899696
PERCISION  large  is : 0.4332434254888739
FPR  large  is : 0.1277355623100304
BER  large  is : 0.23805642212864603
Number of time predicted small  correct : 1399
Number of time predicted small  Wrong : 1831
Number of time predicted different than actual small   : 679
RECALL  small  is : 0.6732435033686237
TNR  small  is : 0.8597364792400797
PERCISION  small  is :

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


...

accuracy feature:bag of words,rating and category for C=  2 0.7066481628337299
Number of time predicted fit  correct : 8025
Number of time predicted fit  Wrong : 942
Number of time predicted different than actual fit   : 3057
RECALL  fit  is : 0.7241472658364916
TNR  fit  is : 0.7674074074074074
PERCISION  fit  is : 0.8949481431917029
FPR  fit  is : 0.2325925925925926
BER  fit  is : 0.2542226633780505
Number of time predicted large  correct : 1273
Number of time predicted large  Wrong : 1686
Number of time predicted different than actual large   : 699
RECALL  large  is : 0.6455375253549696
TNR  large  is : 0.8718844984802432
PERCISION  large  is : 0.4302129097668131
FPR  large  is : 0.12811550151975684
BER  large  is : 0.24128898808239363
Number of time predicted small  correct : 1395
Number of time predicted small  Wrong : 1811
Number of time predicted different than actual small   : 683
RECALL  small  is : 0.6713185755534168
TNR  small  is : 0.8612685766814769
PERCISION  small  is : 

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


accuracy feature:bag of words,rating and category for C=  5 0.7062516521279408
Number of time predicted fit  correct : 8017
Number of time predicted fit  Wrong : 942
Number of time predicted different than actual fit   : 3065
RECALL  fit  is : 0.7234253744811406
TNR  fit  is : 0.7674074074074074
PERCISION  fit  is : 0.8948543364214756
FPR  fit  is : 0.2325925925925926
BER  fit  is : 0.254583609055726
Number of time predicted large  correct : 1273
Number of time predicted large  Wrong : 1689
Number of time predicted different than actual large   : 699
RECALL  large  is : 0.6455375253549696
TNR  large  is : 0.8716565349544073
PERCISION  large  is : 0.4297771775827144
FPR  large  is : 0.12834346504559271
BER  large  is : 0.24140296984531157
Number of time predicted small  correct : 1397
Number of time predicted small  Wrong : 1814
Number of time predicted different than actual small   : 681
RECALL  small  is : 0.6722810394610202
TNR  small  is : 0.8610387620652673
PERCISION  small  is : 0

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


accuracy feature:bag of words,rating and category for C=  10 0.7047316944224161
Number of time predicted fit  correct : 7978
Number of time predicted fit  Wrong : 928
Number of time predicted different than actual fit   : 3104
RECALL  fit  is : 0.7199061541238043
TNR  fit  is : 0.7708641975308642
PERCISION  fit  is : 0.8958005838760387
FPR  fit  is : 0.2291358024691358
BER  fit  is : 0.25461482417266573
Number of time predicted large  correct : 1284
Number of time predicted large  Wrong : 1704
Number of time predicted different than actual large   : 688
RECALL  large  is : 0.6511156186612576
TNR  large  is : 0.870516717325228
PERCISION  large  is : 0.42971887550200805
FPR  large  is : 0.12948328267477205
BER  large  is : 0.23918383200675722
Number of time predicted small  correct : 1402
Number of time predicted small  Wrong : 1836
Number of time predicted different than actual small   : 676
RECALL  small  is : 0.6746871992300288
TNR  small  is : 0.8593534548797304
PERCISION  small  is 

In [130]:
pred_valid= mod1.predict(Xtest)
correct2 = pred_valid==Y_test
print("accuracy feature:bag of words", sum(correct2)/len(correct2))
#print(len(correct1))
print('done..')

accuracy feature:bag of words 0.8018107322231034
done..


In [131]:
pred_valid= mod1.predict(Xtest)
correct2 = pred_valid==Y_test
print("accuracy feature:bag of words", sum(correct2)/len(correct2))
#print(len(correct1))
print('done..')

accuracy feature:bag of words 0.8018107322231034
done..


In [132]:
pred_valid= mod1.predict(Xtest)
correct2 = pred_valid==Y_test
print("accuracy feature:bag of words", sum(correct2)/len(correct2))
#print(len(correct1))
print('done..')

accuracy feature:bag of words 0.8018107322231034
done..


In [133]:
pred_valid= mod1.predict(Xtest)
correct2 = pred_valid==Y_test
print("accuracy feature:bag of words", sum(correct2)/len(correct2))
#print(len(correct1))
print('done..')

accuracy feature:bag of words 0.8018107322231034
done..


In [134]:
pred_valid= mod1.predict(Xtest)
correct2 = pred_valid==Y_test
print("accuracy feature:bag of words", sum(correct2)/len(correct2))
#print(len(correct1))
print('done..')

accuracy feature:bag of words 0.8018107322231034
done..


In [135]:
pred_valid= mod1.predict(Xtest)
correct2 = pred_valid==Y_test
print("accuracy feature:bag of words", sum(correct2)/len(correct2))
#print(len(correct1))
print('done..')

accuracy feature:bag of words 0.8018107322231034
done..


In [136]:
pred_valid= mod1.predict(Xtest)
correct2 = pred_valid==Y_test
print("accuracy feature:bag of words", sum(correct2)/len(correct2))
#print(len(correct1))
print('done..')

accuracy feature:bag of words 0.8018107322231034
done..


In [137]:
dic1= defaultdict(int)
dic2=defaultdict(int)

#print(l[0])
for i in pred_valid:                                       #num of fit/small/large
    j=str(i)
    dic1[j]+=1
for i in Y_test:                                       #num of fit/small/large
    j=str(i)
    dic2[j]+=1
    
print(dic1,dic2)

defaultdict(<class 'int'>, {'fit': 12860, 'large': 1125, 'small': 1147}) defaultdict(<class 'int'>, {'large': 1972, 'fit': 11082, 'small': 2078})


In [138]:
dic1= defaultdict(int)
dic2=defaultdict(int)
dic3=defaultdict(int)
for i,j in zip(pred_valid,Y_test):
    if(i==j and i =='small'):
        dic1[i]+=1
    elif (i==j and i =='large'):
        dic1[i]+=1
    elif(i==j and i =='fit'):
        dic1[i]+=1
    elif(i=='small'):
        dic1['predict small but wrong']+=1
    elif(i=='large'):
        dic1['predicted large but wrong']+=1
    else:
        dic1['predict fit but wrong']+=1
    

In [139]:
print(dic1,dic2)

defaultdict(<class 'int'>, {'predict fit but wrong': 2315, 'fit': 10545, 'large': 804, 'small': 784, 'predicted large but wrong': 321, 'predict small but wrong': 363}) defaultdict(<class 'int'>, {})


In [140]:
j=0
for i in data1:
    if int(i['size']) > j:
        print(i['size'])
        j=int(i['size'])
    

8
12
32
39
45
57
58
